# Atividade 7(a)
* **Nome:** Matheus Freitas Martins 
* **Matrícula:** ES111281

# Importando dependências

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Carregando dados

In [100]:
data = pd.read_csv('breastcancerwisconsin.csv')

# Preparando os dados

In [101]:
# Eliminar coluna 'Unnamed: 32' e 'id'
data = data.drop(['Unnamed: 32', 'id'], axis=1)

# Mapear a coluna 'diagnosis' para valores numéricos
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

# Dividindo os dados em conjuntos de treinamento e teste

In [102]:
# treino (70%), validação (15%) e teste (15%)
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Criando pipeline para cada algoritmo

In [103]:
log_reg_pipeline = Pipeline([('scaler', StandardScaler()), ('log_reg', LogisticRegression(random_state=42))])
rand_forest_pipeline = Pipeline([('scaler', StandardScaler()), ('rand_forest', RandomForestClassifier(random_state=42))])

# Ajustando os hiperparâmetros para Regressão Logística e Random Forest

In [104]:
# log_reg__C: Controla a força da regularização, ajudando a evitar overfitting. Valores menores resultam em maior penalidade e regularização mais forte.
## default: 1.0
# log_reg__solver: Define o algoritmo de otimização utilizado no treinamento do modelo de Regressão Logística.
## default: 'lbfgs'
# log_reg__max_iter: refere-se ao número máximo de iterações que o algoritmo de otimização deve executar antes de parar. 
## default: 100
log_reg_param_grid = {
    'log_reg__C': np.logspace(-4, 4, 10),
    'log_reg__solver': ['newton-cg', 'lbfgs'],
    'log_reg__max_iter': [1000, 5000, 10000]
}


# rand_forest__n_estimators: Controla o número de árvores de decisão na floresta aleatória, afetando a capacidade de generalização e o tempo de treinamento.
## default: 100
# rand_forest__max_depth: Controla a profundidade máxima das árvores de decisão, ajudando a evitar overfitting e influenciando a capacidade do modelo de aprender padrões complexos.
## default: none 
# rand_forest__min_samples_split: Controla o número mínimo de amostras necessárias para dividir um nó interno, ajudando a evitar overfitting e influenciando a capacidade do modelo de aprender padrões complexos.
## default: 2
# rand_forest__min_samples_leaf: Controla o número mínimo de amostras necessárias para formar um nó terminal (folha) 
## default: 1
# rand_forest__max_features: Determina o número de recursos considerados ao procurar a melhor divisão em cada nó da árvore
## default: auto
rand_forest_param_grid = {'rand_forest__n_estimators': [10, 50, 100, 200],
                          'rand_forest__max_depth': [None, 10, 30, 50],
                          'rand_forest__min_samples_split': [2, 5, 10],
                          'rand_forest__min_samples_leaf': [1, 2, 4],
                          'rand_forest__max_features': ['sqrt']}




# Buscando os melhores hiperparâmetros + validação cruzada

In [105]:
log_reg_grid = GridSearchCV(log_reg_pipeline, log_reg_param_grid, cv=5)
rand_forest_grid = GridSearchCV(rand_forest_pipeline, rand_forest_param_grid, cv=5)

# Treinando e avaliando os modelos

In [106]:
log_reg_grid.fit(X_train, y_train)
rand_forest_grid.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros encontrados
print('Melhores hiperparâmetros para Regressão Logística:', log_reg_grid.best_params_)
print('\nMelhores hiperparâmetros para Random Forest:', rand_forest_grid.best_params_)

# acurácia dos modelos otimizados
log_reg_acc = accuracy_score(y_val, log_reg_grid.predict(X_val))
rand_forest_acc = accuracy_score(y_val, rand_forest_grid.predict(X_val))

print('\nAcurácia da Regressão Logística otimizada:', log_reg_acc)
print('Acurácia do Random Forest otimizado:', rand_forest_acc)


Melhores hiperparâmetros para Regressão Logística: {'log_reg__C': 2.782559402207126, 'log_reg__max_iter': 1000, 'log_reg__solver': 'newton-cg'}

Melhores hiperparâmetros para Random Forest: {'rand_forest__max_depth': None, 'rand_forest__max_features': 'sqrt', 'rand_forest__min_samples_leaf': 2, 'rand_forest__min_samples_split': 2, 'rand_forest__n_estimators': 200}

Acurácia da Regressão Logística otimizada: 0.9882352941176471
Acurácia do Random Forest otimizado: 0.9647058823529412
